In [23]:
import tensorflow.keras.layers as tkl

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD

In [24]:
# tf.config.experimental_run_functions_eagerly(True)
class Res_BLock(tkl.Layer):
    def __init__(self, filters: int):
        super(Res_BLock, self).__init__()
        self.con1 = tkl.Conv2D(filters=64, kernel_size=(3, 3),
                               padding="same")
        self.con2 = tkl.Conv2D(filters=filters, kernel_size=(3, 3),
                               padding="same")
        self.act1 = tkl.ReLU()
        self.act2 = tkl.ReLU()
        self.sample = Sequential()
        self.sample.add(tkl.Conv2D(filters=filters, kernel_size=(1, 1)))

    def call(self, inputs, *args, **kwargs):
        res = self.sample(inputs)
        output = self.con1(inputs)
        output = self.act1(output)
        output = self.con2(output)

        output = tkl.add([output, res])
        output = self.act2(output)
        return output

class ResNet(Model):
    def get_config(self):
        super(ResNet, self).get_config()

    def __init__(self):
        super(ResNet, self).__init__()
        self.block1 = Sequential()
        self.block1.add(tkl.Conv2D(filters=64, kernel_size=(3, 3), padding="same"))
        self.block1.add(tkl.Activation("relu"))
        self.block1.add(tkl.BatchNormalization())
        self.block1.add(tkl.MaxPooling2D(pool_size=(3, 3), strides=2, padding="same"))

        self.res_mod = Sequential([Res_BLock(128)])

        self.block2 = Sequential()
        self.block2.add(tkl.BatchNormalization())
        self.block2.add(tkl.AvgPool2D(pool_size=(3, 3), strides=1, padding="same"))

        self.res_mod2 = Sequential([Res_BLock(128)])

        self.block3 = Sequential()
        self.block3.add(tkl.AvgPool2D(pool_size=(5, 5), strides=1, padding="same"))
        self.block3.add(tkl.Dropout(0.5))

        self.block3.add(tkl.Flatten())
        self.block3.add(tkl.Dense(518))
        self.block3.add(tkl.Activation("relu"))
        self.block3.add(tkl.Dense(2))
        self.block3.add(tkl.Activation("softmax"))


    def call(self, inputs, training=None, mask=None):
        output = self.block1(inputs)
        output = self.res_mod(output)
        output = self.block2(output)
        output = self.res_mod2(output)
        output = self.block3(output)
        return output

In [25]:
model = ResNet()
model.build(input_shape=(None, 64, 64, 3))
model.summary()

Model: "res_net_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_44 (Sequential)   (None, 32, 32, 64)        2048      
_________________________________________________________________
sequential_46 (Sequential)   (None, 32, 32, 128)       119104    
_________________________________________________________________
sequential_47 (Sequential)   (None, 32, 32, 128)       512       
_________________________________________________________________
sequential_49 (Sequential)   (None, 32, 32, 128)       164160    
_________________________________________________________________
sequential_50 (Sequential)   (None, 2)                 67896852  
Total params: 68,182,676
Trainable params: 68,182,292
Non-trainable params: 384
_________________________________________________________________


In [26]:
train_dir = "data/training_set"
test_dir = "data/test_set"
data_gen = ImageDataGenerator(rescale=1.0 / 255)
train_data = data_gen.flow_from_directory(train_dir,
                                          target_size=(64, 64))
test_data = data_gen.flow_from_directory(test_dir,
                                         target_size=(64, 64))
print("数据加载完成")
print(train_data.class_indices)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
数据加载完成
{'cats': 0, 'dogs': 1}


In [27]:
epochs = 10
sgd = SGD(learning_rate=0.01,
          momentum=0.05,
          nesterov=True,
          )
model.compile(loss="binary_crossentropy",
              optimizer=sgd,
              metrics=["acc"])
print("模型编译成功")
model.summary()

模型编译成功
Model: "res_net_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_44 (Sequential)   (None, 32, 32, 64)        2048      
_________________________________________________________________
sequential_46 (Sequential)   (None, 32, 32, 128)       119104    
_________________________________________________________________
sequential_47 (Sequential)   (None, 32, 32, 128)       512       
_________________________________________________________________
sequential_49 (Sequential)   (None, 32, 32, 128)       164160    
_________________________________________________________________
sequential_50 (Sequential)   (None, 2)                 67896852  
Total params: 68,182,676
Trainable params: 68,182,292
Non-trainable params: 384
_________________________________________________________________


In [28]:
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data,
                    batch_size=64)

Epoch 1/10
251/251 [==============================] - 63s 236ms/step - loss: 0.6491 - acc: 0.6496 - val_loss: 0.9280 - val_acc: 0.4998